In [ ]:
import time
import os  
from bs4 import BeautifulSoup as bs
import pandas as pd
import scipy as sc
import numpy as np
import requests
from lxml import html
import csv
import matplotlib.pyplot as plt

In [ ]:
#Crawling

In [ ]:
def load_soup_object(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
    r = requests.get(url,headers=headers)
    soup = bs(r.content, "html.parser")
    return soup

In [ ]:
i = 0
pages = []
while(i<350): ## 350
    i = i +1
    i_str = str(i)
    url = 'https://www.coffeereview.com/review/page/'+i_str+'/'
    pages.append(url)


In [ ]:
pages

In [ ]:
innerPagesLinks = []
for page in pages:
    soup = load_soup_object(page)
    items = soup.find_all("div",{"class": "entry-content"})
    for item in items:
        try:
            link = item.find("p",{"class": "review-roaster"}).find("a")["href"]
        except:
            link = None
        if link:
            innerPagesLinks.append(link)

In [ ]:
innerPagesLinks

In [ ]:
data=[]

for page in innerPagesLinks:
    soup = load_soup_object(page)
    dataList = soup.find("div",{"class": "review-template"})
    try:
        coffee_name = dataList.find("p",{"class":"review-roaster"}).text.strip()
    except:
        coffee_name = None
    try:
        company_name = dataList.find("h1",{"class":"review-title"}).text.strip()
    except:
        company_name = None
        
    try:
        rating = dataList.find("span",{"class":"review-template-rating"}).text.strip()
    except:
        rating = None
        
    try:
        location = dataList.find("table",{"class":"review-template-table"})("tr")[1]("td")[1].text.strip()
    except:
        location = None
        
    try:
        r_location = dataList.find("table",{"class":"review-template-table"})("tr")[0]("td")[1].text.strip()
    except:
        r_location = None
        
    try:
        roast = dataList.find("table",{"class":"review-template-table"})("tr")[2]("td")[1].text.strip()
    except:
        roast = None
        
    try:
        agtron = dataList.find("table",{"class":"review-template-table"})("tr")[3]("td")[1].text.strip()
    except:
        agtron = None
        
    try:
        price = dataList.find("table",{"class":"review-template-table"})("tr")[4]("td")[1].text.strip()
    except:
        price = None
        
    try:
        date = dataList.findAll("table",{"class":"review-template-table"})[1]("tr")[0]("td")[1].text.strip()
    except:
        date = None
        
    try:
        aroma = dataList.findAll("table",{"class":"review-template-table"})[1]("tr")[1]("td")[1].text.strip()
    except:
        aroma = None
    
    try:
        acidity = dataList.findAll("table",{"class":"review-template-table"})[1]("tr")[2]("td")[1].text.strip()
    except:
        acidity = None
        
    try:
        body = dataList.findAll("table",{"class":"review-template-table"})[1]("tr")[3]("td")[1].text.strip()
    except:
        body = None
    
    try:
        flavor = dataList.findAll("table",{"class":"review-template-table"})[1]("tr")[4]("td")[1].text.strip()
    except:
        flavor = None
    
    try:
        aftertaste = dataList.findAll("table",{"class":"review-template-table"})[1]("tr")[5]("td")[1].text.strip()
    except:
        aftertaste = None
        
    rowData={ "Coffee Name":coffee_name,"Company Name":company_name, "Origin country":location, "Roaster Location:":r_location, "Roast Level": roast, "Agtron":agtron, "Price":price, "Rating":rating, "Review Date":date, "Aroma":aroma, "Acidity":acidity, "Body":body, "Flavor":flavor, "Aftertaste":aftertaste}
    data.append(rowData)

time.sleep(10)
df= pd.DataFrame(data)
fileName='coffee.csv'
df.to_csv(fileName, encoding='utf-8', index=False)

In [ ]:
df=pd.read_csv('coffee.csv')

In [ ]:
df

In [ ]:
#Cleaning

In [ ]:
df=pd.read_csv('coffee.csv')

In [ ]:
dfclean1=df.drop_duplicates().copy() #נזרוק את כל השורות הכפולות

In [ ]:
dfclean1

In [ ]:
print(dfclean1['Rating'].isnull().sum())
print(dfclean1['Company Name'].isnull().sum())
print(dfclean1.Agtron.isnull().sum())
print(dfclean1.Aroma.isnull().sum())
print(dfclean1.Price.isnull().sum())
print(dfclean1.Acidity.isnull().sum())
print(dfclean1.Body.isnull().sum())
print(dfclean1.Flavor.isnull().sum())
print(dfclean1.isnull()['Origin country'].sum())
print(dfclean1.isnull()['Roast Level'].sum())
print(dfclean1.isnull()['Roaster Location:'].sum())
print(dfclean1.isnull()['Review Date'].sum())
print(dfclean1.Aftertaste.isnull().sum())

In [ ]:
#נתחיל לנקות פר משתנה כשבכל פעם שנעבור משתנה ניצור עותק  
dfclean1.Aftertaste.unique()

In [ ]:
#נמיר למשתנה נומרי 
dfclean1.Aftertaste=dfclean1.Aftertaste.astype(float)
dfclean1

In [ ]:
avg=dfclean1.Aftertaste.mean()
dfclean1.Aftertaste.fillna(avg, inplace=True) 
dfclean2=dfclean1.copy()

#נמלא את כל התיבות הריקות בערך הממוצע

In [ ]:
#משתנה שני- price
import re

def convert_to_dollars(value):
    # Extract dollar amount
    match = re.search('\$([\d\.]+)', str(value))
    if match:
        dollars = match.group(1)
    else:
        dollars = None
    return dollars

dfclean2['Price'] = dfclean2['Price'].apply(convert_to_dollars).astype(float)

In [ ]:
dfclean2= dfclean2[dfclean2['Price'].notna()] #נמחוק את השורות שלא הוזן מחיר

In [ ]:
print(dfclean2['Rating'].isnull().sum())
print(dfclean2['Company Name'].isnull().sum())
print(dfclean2.Agtron.isnull().sum())
print(dfclean2.Aroma.isnull().sum())
print(dfclean2.Price.isnull().sum())
print(dfclean2.Acidity.isnull().sum())
print(dfclean2.Body.isnull().sum())
print(dfclean2.Flavor.isnull().sum())
print(dfclean2.isnull()['Origin country'].sum())
print(dfclean2.isnull()['Roast Level'].sum())
print(dfclean2.isnull()['Roaster Location:'].sum())
print(dfclean2.isnull()['Review Date'].sum())
print(dfclean2.Aftertaste.isnull().sum())

In [ ]:
dfclean3=dfclean2.copy()

In [ ]:
dfclean3.Flavor.unique() #משתנה שלישי-flavor

In [ ]:
dfclean3.Flavor=dfclean3.Flavor.astype(float)
avg=dfclean3.Flavor.mean()
dfclean3.Flavor.fillna(avg, inplace=True)
dfclean3.info()
dfclean4=dfclean3.copy()

In [ ]:
dfclean4.Body.unique() #משתנה רביעי -body

In [ ]:
dfclean4.Body=dfclean4.Body.astype(float)
avg=dfclean4.Body.mean()
dfclean4.Body.fillna(avg, inplace=True)
dfclean4.info()
dfclean5=dfclean4.copy()

In [ ]:
dfclean5.Acidity.unique() #משתנה חמישי- acidity

In [ ]:
dfclean5.Acidity=dfclean5.Acidity.astype(float)
avg=dfclean5.Acidity.mean()
dfclean5.Acidity.fillna(avg, inplace=True)
dfclean5.info()
dfclean6=dfclean5.copy()

In [ ]:
dfclean6.Aroma.unique() #משתנה שישי- aroma

In [ ]:
dfclean6.Aroma=dfclean6.Aroma.astype(float)
avg=dfclean6.Aroma.mean()
dfclean6.Aroma.fillna(avg, inplace=True)
dfclean6.info()
dfclean6
dfclean7=dfclean6.copy()

In [ ]:
print(dfclean6['Rating'].unique())
print(dfclean6['Company Name'].unique())
print(dfclean6.Agtron.isnull().unique())
print(dfclean6.Aroma.unique())
print(dfclean6.Price.unique())
print(dfclean6.Acidity.unique())
print(dfclean6.Body.unique())
print(dfclean6.Flavor.unique())
print(dfclean6['Origin country'].unique())
print(dfclean6['Roast Level'].unique())
print(dfclean6['Roaster Location:'].unique())
print(dfclean6['Review Date'].unique())
print(dfclean6.Aftertaste.unique())

In [ ]:
#משתנה שביעי- roast level
#מחרוזות של מלל יותר קשה להמיר למשתנה נומרי, מצריכות ליצור מילון וידנית לתת ערך מספרי לכל אחת מהאפשרויות
dfclean7.Rating=dfclean7.Rating.astype(float)
roast_level_dict = {'Light': 1, 'Medium-Light': 2, 'Medium': 3, 'Medium-Dark': 4, 'Dark': 5, 'NA': 6, 'Very Dark': 7}

# Use the map function to apply the dictionary to the column values
dfclean7['Roast Level'] = dfclean7['Roast Level'].map(roast_level_dict)

# Verify the result
print(dfclean7['Roast Level'].unique())
dfclean8=dfclean7.copy()

In [ ]:
dfclean8.Agtron.unique()#משתה שמיני-Agtron

In [ ]:
dfclean8 = dfclean8[dfclean8['Agtron'] != 'NA/']
dfclean8 = dfclean8[dfclean8['Agtron'] != 'NA/NA']
dfclean8 = dfclean8[dfclean8['Agtron'] != '/']
dfclean8 = dfclean8[dfclean8['Agtron'] != '0/0']
dfclean8=dfclean8.copy()

In [ ]:
dfclean8['Agtron']=dfclean8.Agtron.apply(lambda x: eval(x) if isinstance(x, str) and '0' not in x.split('/') else x)
dfclean8['Agtron'] = dfclean8['Agtron'].replace('/', 0, regex=True)
dfclean8['Agtron'].unique()

In [ ]:
dfclean8['Agtron'] = dfclean8['Agtron'].astype(float)

In [ ]:
dfclean8.info()

In [ ]:
dfclean9=dfclean8.copy()

In [ ]:
dfclean9['Review Date'] = dfclean9['Review Date'].str[-4:].astype(int) #משתנה תשיעי -reviwe date

In [ ]:
dfclean10=dfclean9.copy()

In [ ]:
dfclean10['Roaster Location:'].value_counts() #משתנה עשירי-roaster location

In [ ]:
dfclean10['Roaster Location:'].unique()

In [ ]:
dfclean10['Roaster Location:'] = dfclean10['Roaster Location:'].apply(lambda x: x.split(", ")[-1]) #נפצל את הערים מהמדינות בהן נמצאות


In [ ]:
#ננקה את הכפילויות העקומות, (הוכחה שזה לא הג'יפיטי)
dfclean10['Roaster Location:'] = dfclean10['Roaster Location:'].replace({
    'Los Angeles': 'California',
    'Wisconsin.': 'Wisconsin',
    'Taiwan.': 'Taiwan',
    'Montana.': 'Montana',
    'New York.': 'New York',
    'Calfornia': 'California',
    'California.': 'California',
    'Branford Connecticut': 'Connecticut',
    'Vermont.': 'Vermont',
    'Illinois.': 'Illinois',
    'MIssissippi': 'Mississippi',
    'MIssouri': 'Missouri',
    'D.C.': 'DC',
    'MInnesota': 'Minnesota',
    "Hawai'i": 'Hawaii',
    'Hawai’i': 'Hawaii',
    'Peoria. Illinois': 'Illinois',
    'Virginia.':'Virginia'
})

In [ ]:
dfclean10['Roaster Location:'].unique()

In [ ]:
 data_dict = dfclean10['Roaster Location:'].value_counts().to_dict()
data_dict
 # merge the least common locations (under 1%) into a single category called "Other".
threshold =50
other_count = sum(count for count in data_dict.values() if count < threshold)
data_dict = {k: v for k, v in data_dict.items() if v >= threshold}
data_dict['Other'] = other_count

# create a pie chart from the updated dictionary
plt.figure(figsize=(8, 8))
plt.pie(data_dict.values(), labels=data_dict.keys(),autopct='%1.1f%%')

# show the pie chart
plt.show()

In [ ]:
# create a dictionary with mappings
location_dict = {'Virginia': 1, 'Vermont': 2, 'Colorado': 3, 'Wyoming': 4, 'Oregon': 5, 'Montana': 6, 
                 'Utah': 7, 'California': 8, 'Kansas': 9, ' California': 8, 'Arizona': 10, 'Hawaii': 11, 
                 'Taiwan': 12, 'Wisconsin': 13, 'Illinois': 14, 'Minnesota': 15, 'Ohio': 16, 'Georgia': 17, 
                 'Maine': 18, 'China': 19, 'North Carolina': 20, 'New York': 21, 'Connecticut': 22, 'Alaska': 23, 
                 'Guatemala': 24, 'New Jersey': 25, 'Florida': 26, 'Japan': 27, 'Nevada': 28, 'Canada': 29, 
                 'Texas': 30, 'Michigan': 31, 'Massachusetts': 32, 'Oklahoma': 33, 'Kenya': 34, 'New Mexico': 35, 
                 'Washington': 36, 'New Hampshire': 37, 'Pennsylvania': 38, 'Alabama': 39, 'DC': 40, 
                 'Colombia': 41, 'Tennessee': 42, 'Mexico': 43, 'Honduras': 44, 'Peru': 45, 'Missouri': 46, 
                 'Mississippi': 47, 'Iowa': 48, 'Maryland': 49, 'Washingto': 36, 'Laos': 50, 'Indonesia': 51, 
                 'Australia': 52, 'United Arab Emirates': 53, 'Louisiana': 54, 'Thailand': 55, 'Uganda': 56, 
                 'Idaho': 57, 'Californiaa': 8, 'South Korea': 58, 'Kentucky': 59, 
                 'Democratic Republic of the Congo': 60, 'Rwanda': 61, 'Puerto Rico': 62, 'Tempe Arizona': 10, 
                 'Germany': 63, 'El Salvador': 64, 'Indiana': 65, 'Italy': 66, 'Switzerland': 67, 'Costa Rica': 68, 
                 'Sacr': 69, 'Kona': 70, 'Nebraska': 71, 'South Dakota': 72, 'Sweden': 73, 'Arkansas': 74, 
                 'British Colombia': 75, 'France': 76, 'Rhode Island': 77}

# apply the mapping to the column using the map() method
dfclean10['Roaster Location:'] = dfclean10['Roaster Location:'].map(location_dict)

In [ ]:
dfclean10['Roaster Location:'].unique()

In [ ]:
data_dict = dfclean10['Roaster Location:'].value_counts().to_dict()
data_dict
 # merge the least common locations (under 1%) into a single category called "Other", there are 24 in total.
threshold =55
other_count = sum(count for count in data_dict.values() if count < threshold)
data_dict = {k: v for k, v in data_dict.items() if v >= threshold}
data_dict['Other'] = other_count

# create a pie chart from the updated dictionary
plt.figure(figsize=(8, 8))
plt.pie(data_dict.values(), labels=data_dict.keys(),autopct='%1.1f%%')

# show the pie chart
plt.show()

In [ ]:
 dfclean10. info()

In [ ]:
fileName='coffeeClean2'

In [ ]:
dfclean10.to_csv(fileName, encoding='utf-8', index=False)

In [ ]:
#visualization-EDA

In [ ]:
#EDA- part1
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

#EDA- part2
import matplotlib.image as mpimg
from IPython.display import IFrame

In [ ]:
# Top ratings are: 93, 92, 94, 95, 90
df["Rating"].value_counts()

In [ ]:
# Presented in bar & pie graph
df["Rating"].value_counts().plot(kind='bar')

In [ ]:
data_dict = df['Rating'].value_counts().to_dict()
data_dict

# create a pie chart from the updated dictionary
plt.figure(figsize=(8, 8))
plt.pie(data_dict.values(), labels=data_dict.keys(),autopct='%1.1f%%')

# show the pie chart
plt.show()

In [ ]:
# Distribution of reviews by coffee harvest Origin country
df['Origin country'].value_counts()

In [ ]:
# Distribution of reviews by dates 
# Most of the reviews are from Oct-Nov.22 
df["Review Date"].head(1000).value_counts().plot(kind='pie')

In [ ]:
# Distribution of reviews by price
df[['Rating', 'Price']].value_counts()

In [ ]:
#Converting Rating to bool, adding a new column "Rating Bool" 1- rating > 90 else -0.

dfeda1 = df.copy()
dfeda1['Rating Bool'] = (dfeda1['Rating'] > 90).astype(int)
dfeda1.tail(1000)

In [ ]:
bins=[0,50,100,200,300,400,500,1000]
labels=[1,2,3,4,5,6,7]

dfeda2 = dfeda1.copy()
dfeda2['Price_Binned']=pd.cut(dfeda2['Price'],bins,labels)

In [ ]:
dfeda2[['Rating Bool', 'Price_Binned']].head(1000).value_counts()

In [ ]:
ct1 = pd.crosstab(dfeda2['Rating Bool'] , dfeda2['Price_Binned'])
ct1.head()

In [ ]:
ct2 = pd.crosstab(dfeda2['Rating Bool'] , dfeda2['Price_Binned'], normalize='index')
ct2.head()

In [ ]:
ct2.plot(kind='bar')
plt.title('Distribution of reviews by price')
plt.xlabel('Rating: (Rating > 90: 1, else: 0)')
plt.ylabel('Price')

In [ ]:
#Correlation between price and rating
from scipy.stats import chi2_contingency
chi2_contingency (ct2)

In [ ]:
dfeda2[['Rating', 'Aftertaste']].head(1000).value_counts()

In [ ]:
ct3 = pd.crosstab(dfeda2['Rating'] , dfeda2['Aftertaste'], normalize='index')
ct3.head()

In [ ]:
ct3.plot(kind='bar')
plt.title('Distribution of reviews by aftertaste')
plt.xlabel('Rating')
plt.ylabel('Aftertaste')

In [ ]:
#Correlation between aftertaste and rating
chi2_contingency (ct3)

In [ ]:
coffee1 = dfeda2.groupby(['Rating']).mean()[['Aroma', 'Price', 'Body', 'Acidity','Aftertaste']]
coffee1.plot()

In [ ]:
sns.scatterplot(x='Price', y='Aroma',  s=15, hue='Rating Bool', data=dfeda2)

In [ ]:
sns.scatterplot(x='Price', y='Flavor',  s=20, hue='Rating', data=dfeda2)

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
dfeda2.describe()

In [ ]:
plt.scatter(x=dfeda2['Flavor'],y=dfeda2['Rating'],c='r',marker='s',label='Flavor')
plt.scatter(x=dfeda2['Aroma'],y=dfeda2['Rating'],c='b',marker='o',label='Aroma')
plt.scatter(x=dfeda2['Body'],y=dfeda2['Rating'],c='k',marker='*',label='Body')
plt.scatter(x=dfeda2['Aftertaste'],y=dfeda2['Rating'],c='m',marker='p',label='Aftertaste')
plt.scatter(x=dfeda2['Acidity'],y=dfeda2['Rating'],c='g',marker='+',label='Acidity')

plt.legend(numpoints=1,loc=4)
plt.title("Rating's correlation")
plt.xlabel('Rank')
plt.ylabel('Rating')
plt.show()

In [ ]:
plt.scatter(x=dfeda2['Aroma'],y=dfeda2['Rating'],c='b',marker='o',label='Aroma')

plt.legend(numpoints=1,loc=4)
plt.title('Distribution of Rating by Aroma')
plt.xlabel('Aroma')
plt.ylabel('Rating')
plt.show()

In [ ]:
sns.pairplot(dfeda2[['Rating', 'Price', 'Aroma', 'Aftertaste', 'Body', 'Flavor']])

In [ ]:
#Machine learning

In [ ]:
dfclean=pd.read_csv('coffeeClean2')

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression


#and visualization setup
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6)
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
dfclean.info()

In [ ]:
dfclean.columns[(dfclean.columns != 'Rating') & (dfclean.columns != 'Company Name') & (dfclean.columns != 'Coffee Name') & (dfclean.columns != 'Origin country')]

In [ ]:
#creating matrix X and vector Y
#מנסה לנרמל לפני פיצול 
TRAINING_FEATURES = dfclean.columns[(dfclean.columns != 'Rating') & (dfclean.columns != 'Company Name') & (dfclean.columns != 'Coffee Name') & (dfclean.columns != 'Origin country')]
TARGET_FEATURE    = 'Rating'

X = dfclean[TRAINING_FEATURES]
y = dfclean[TARGET_FEATURE]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#min_max_scaler = MinMaxScaler(feature_range=(0, 100))
#X_train_scaled_in_range = min_max_scaler.fit_transform(X_train)
#print("Min Value: ", X_train_scaled_in_range.min(axis=0))
#print("Max Value: ", X_train_scaled_in_range.max(axis=0))
#X_test_scaled_in_range = min_max_scaler.transform(X_test)
#X_test_scaled_in_range

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

In [ ]:
print("Initial amount of samples: #{}".format(X.shape[0]))
print("Number of training samples: #{}".format(X_train.shape[0]))
print("Number of test samples: #{}".format(X_test.shape[0]))

print("\nTarget distribution in original dataset:\n{}".format(y.value_counts()))
print("\nTarget distribution in the training set:\n{}\n".format(y_train.value_counts()))
print("Target distribution in the test set:\n{}".format(y_test.value_counts()))

In [ ]:
X.head()

In [ ]:
X_train

In [ ]:
#יצירת מודל רגרסיה ליניארית ובדיקתו במבחן R2
lr=linear_model.LinearRegression()
lr.fit(X=X_train, y=y_train)
print("Slope:",lr.coef_)
print("Intercept:",lr.intercept_)
print("R2:",lr.score(X_train,y_train))

print('linear regression train accuracy %s'% lr.score(X_train,y_train))
print('linear regression test accuracy %s'% lr.score(X_test,y_test))

In [ ]:
y_pred=lr.predict(X_test)
dfclean['Rating']

In [ ]:
dfclean1=pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
dfclean1

In [ ]:
X.columns[lr.coef_.argmax()]

In [ ]:
import matplotlib.pyplot as plt

# create a scatter plot with the actual and predicted values
plt.scatter(range(len(y_test)), y_test, color='red', label='Actual Rating')
plt.scatter(range(len(y_pred)), y_pred, color='blue', label='Predicted Rating')
plt.xlabel('Index')
plt.ylabel('Rating')
plt.title('Linear Regression Model')
plt.legend()
plt.show()

In [ ]:
#ננסה עוד מודל ,כיוון שהתוצאות יצאו גבוהות ונרצה לודא אמינות 
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=45)
model = RandomForestRegressor(n_estimators=100, random_state=45).fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))

In [ ]:
dfclean.corr()
#ניתן לראות שהקאורלציה של רייטינג מול משתנים אחרים היא יחסית גבוהה . בעיקר בפלייבור . אפשר לעשות גם מטריצת עמודות לדבר הזה 

In [ ]:
# create a scatter plot with the actual and predicted values
plt.scatter(range(len(y_test)), y_test, color='red', label='Actual Rating')
plt.scatter(range(len(y_pred)), y_pred, color='blue', label='Predicted Rating')
plt.xlabel('Index')
plt.ylabel('Rating')
plt.title('Linear Regression Model')
plt.legend()
plt.show()